In [9]:
from google.cloud import bigquery

import os
import pandas as pd

In [14]:
# add GCP project and credentials here
YOUR_PROJECT_NAME = 'opensource-observer'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../gcp_credentials.json'

# connect to OSO
client = bigquery.Client()

In [39]:
query = f"""
select distinct
  fid,
  target_fid,
  timestamp
from `{YOUR_PROJECT_NAME}.farcaster.links`
where
  deleted_at is null
  and type = 'follow'
order by timestamp
"""  

result = client.query(query)
df = result.to_dataframe()

In [54]:
P = 4
partition = len(df) // P
j = 0
for i in range(P):
    k = min(j + partition, len(df))
    dff = df.iloc[j:k]
    min_date = dff['timestamp'].min().strftime("%Y%m%d%H%M%S")
    max_date = dff['timestamp'].max().strftime("%Y%m%d%H%M%S")
    dff[['fid', 'target_fid']].to_csv(f"{min_date}_{max_date}_farcaster_graph.csv")
    j = k

In [66]:
query = f"""
select
  user_artifacts.artifact_name as git_user,
  repo_artifacts.artifact_namespace as repo_owner,
  repo_artifacts.artifact_name as repo_name,
  events.event_type,
  sum(amount) as amount
from `{YOUR_PROJECT_NAME}.oso.timeseries_events_by_artifact_v0` as events
join `{YOUR_PROJECT_NAME}.oso.artifacts_v1` as user_artifacts
  on events.from_artifact_id = user_artifacts.artifact_id
join `{YOUR_PROJECT_NAME}.oso.artifacts_v1` as repo_artifacts
  on events.to_artifact_id = repo_artifacts.artifact_id
where
    events.time >= '2023-09-01'
    and event_type in ('COMMIT_CODE', 'STARRED', 'FORKED', 'ISSUE_OPENED', 'ISSUE_COMMENT')
group by 1,2,3,4
"""

result = client.query(query)
df = result.to_dataframe()

In [67]:
len(df)

846954

In [68]:
df.head()

,git_user,repo_owner,repo_name,event_type,amount
0,a7ex12,ethereum,pyethereum,FORKED,1.0
1,sarnaz1304,cyfrin,cyfrin-audit-reports,FORKED,1.0
2,jsunjava,kkrt-labs,kakarot,FORKED,1.0
3,chloe-tan,uniswap,v3-sdk,FORKED,1.0
4,lehuynhphat2808,mofalabs,zklogin,FORKED,1.0


In [69]:
df.to_csv("20230901_20240915_github.csv")

In [70]:
df

,git_user,repo_owner,repo_name,event_type,amount
0,a7ex12,ethereum,pyethereum,FORKED,1.0
1,sarnaz1304,cyfrin,cyfrin-audit-reports,FORKED,1.0
2,jsunjava,kkrt-labs,kakarot,FORKED,1.0
3,chloe-tan,uniswap,v3-sdk,FORKED,1.0
4,lehuynhphat2808,mofalabs,zklogin,FORKED,1.0
...,...,...,...,...,...
846949,codegovindz,huggingface,transformers,ISSUE_COMMENT,1.0
846950,shahad-mahmud,huggingface,transformers,ISSUE_COMMENT,2.0
846951,moghadas76,huggingface,transformers,ISSUE_COMMENT,3.0
846952,zhangjunyi111,huggingface,transformers,ISSUE_COMMENT,1.0


In [71]:
df['event_type'].value_counts()

event_type
STARRED          491327
FORKED           164050
ISSUE_COMMENT     96458
ISSUE_OPENED      54920
COMMIT_CODE       40199
Name: count, dtype: int64